# YouTube Views Predictor - Analysis Notebook

This notebook demonstrates:
1. Loading and exploring the dataset
2. Feature engineering process
3. Model training and evaluation
4. Making predictions
5. Feature importance analysis
6. Optimization recommendations

## 1. Setup and Imports

In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from utils.feature_engineering import FeatureExtractor, get_optimal_features
from utils.model_training import YouTubeViewsPredictor, create_sample_dataset

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

%matplotlib inline

## 2. Load or Create Dataset

In [ ]:
# Create sample dataset
df = create_sample_dataset(n_samples=1000, output_path='../data/processed/sample_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

## 3. Explore the Data

In [ ]:
# Basic statistics
print("View Count Statistics:")
print(df['views'].describe())

# Distribution of views
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(df['views'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Views')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Views')

axes[1].boxplot(df['views'])
axes[1].set_ylabel('Views')
axes[1].set_title('Views Boxplot')

plt.tight_layout()
plt.show()

## 4. Feature Analysis

In [ ]:
# Analyze relationship between features and views
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Peak hour vs views
axes[0, 0].boxplot([df[df['is_peak_hour']==0]['views'], df[df['is_peak_hour']==1]['views']], labels=['Non-Peak', 'Peak'])
axes[0, 0].set_ylabel('Views')
axes[0, 0].set_title('Views by Peak Hour')

# Weekend vs views
axes[0, 1].boxplot([df[df['is_weekend']==0]['views'], df[df['is_weekend']==1]['views']], labels=['Weekday', 'Weekend'])
axes[0, 1].set_ylabel('Views')
axes[0, 1].set_title('Views by Weekend')

# Duration vs views
axes[0, 2].scatter(df['duration_minutes'], df['views'], alpha=0.5)
axes[0, 2].set_xlabel('Duration (minutes)')
axes[0, 2].set_ylabel('Views')
axes[0, 2].set_title('Duration vs Views')

# Title length vs views
axes[1, 0].scatter(df['title_length'], df['views'], alpha=0.5)
axes[1, 0].set_xlabel('Title Length')
axes[1, 0].set_ylabel('Views')
axes[1, 0].set_title('Title Length vs Views')

# Tags count vs views
axes[1, 1].scatter(df['tags_count'], df['views'], alpha=0.5)
axes[1, 1].set_xlabel('Number of Tags')
axes[1, 1].set_ylabel('Views')
axes[1, 1].set_title('Tags Count vs Views')

# Question mark vs views
axes[1, 2].boxplot([df[df['has_question_mark']==0]['views'], df[df['has_question_mark']==1]['views']], labels=['No ?', 'Has ?'])
axes[1, 2].set_ylabel('Views')
axes[1, 2].set_title('Views by Question Mark')

plt.tight_layout()
plt.show()

## 5. Train the Model

In [ ]:
# Prepare features and target
feature_cols = [col for col in df.columns if col != 'views']
X = df[feature_cols]
y = df['views']

print(f"Number of features: {len(feature_cols)}")
print(f"Number of samples: {len(X)}")

In [ ]:
# Train model
predictor = YouTubeViewsPredictor(model_type='xgboost')
results = predictor.train(X, y, test_size=0.2, random_state=42)

print("\nTraining Results:")
print("="*50)
print("\nTraining Metrics:")
for metric, value in results['train_metrics'].items():
    print(f"  {metric.upper()}: {value:.2f}")

print("\nTest Metrics:")
for metric, value in results['test_metrics'].items():
    print(f"  {metric.upper()}: {value:.2f}")

## 6. Feature Importance

In [ ]:
# Get and visualize feature importance
top_features = predictor.get_top_features(n=15)

features = [f[0] for f in top_features]
importances = [f[1] for f in top_features]

plt.figure(figsize=(12, 8))
plt.barh(features, importances)
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Top 15 Most Important Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 10 Features:")
for i, (feature, importance) in enumerate(top_features[:10], 1):
    print(f"{i:2d}. {feature:30s}: {importance:.4f}")

## 7. Make Predictions

In [ ]:
# Create feature extractor
extractor = FeatureExtractor()

# Example video 1: Optimal parameters
video_optimal = {
    'title': 'How I Built This Amazing Project in 10 Minutes! 🚀',
    'duration': 600,  # 10 minutes
    'tags': 'tutorial,programming,python,coding,beginners,project,howto,learn,tips,tricks',
    'publish_time': '2024-01-15 19:00:00',  # 7 PM, Friday
    'description': 'In this comprehensive tutorial, learn how to build an amazing project step by step. Subscribe for more content! Check out my website: https://example.com'
}

# Example video 2: Non-optimal parameters
video_suboptimal = {
    'title': 'video',
    'duration': 120,  # 2 minutes
    'tags': 'vid',
    'publish_time': '2024-01-08 09:00:00',  # 9 AM, Monday
    'description': 'short desc'
}

# Extract features and predict
features_optimal = extractor.extract_all_features(video_optimal)
features_suboptimal = extractor.extract_all_features(video_suboptimal)

pred_optimal = predictor.predict(features_optimal)[0]
pred_suboptimal = predictor.predict(features_suboptimal)[0]

print("Prediction Results:")
print("="*50)
print(f"\nOptimal Video:")
print(f"  Predicted Views: {pred_optimal:,.0f}")
print(f"\nSub-optimal Video:")
print(f"  Predicted Views: {pred_suboptimal:,.0f}")
print(f"\nDifference: {pred_optimal - pred_suboptimal:,.0f} views")
print(f"Improvement: {((pred_optimal / pred_suboptimal - 1) * 100):.1f}%")

## 8. Optimization Analysis

In [ ]:
# Analyze impact of different publish hours
base_video = {
    'title': 'Great Tutorial for Beginners',
    'duration': 600,
    'tags': 'tutorial,howto,learn,tips,guide,beginners,easy,simple,quick,best',
    'publish_time': '2024-01-15 12:00:00',
    'description': 'Learn something useful in this tutorial. Subscribe for more great content!'
}

# Test different hours
hours = list(range(24))
predictions_by_hour = []

for hour in hours:
    test_video = base_video.copy()
    test_video['publish_time'] = f'2024-01-15 {hour:02d}:00:00'
    features = extractor.extract_all_features(test_video)
    pred = predictor.predict(features)[0]
    predictions_by_hour.append(pred)

# Plot
plt.figure(figsize=(14, 6))
plt.plot(hours, predictions_by_hour, marker='o', linewidth=2, markersize=8)
plt.xlabel('Publish Hour (24h format)')
plt.ylabel('Predicted Views')
plt.title('Impact of Publishing Hour on Predicted Views')
plt.grid(True, alpha=0.3)
plt.xticks(hours)

# Highlight peak hours
plt.axvspan(18, 21, alpha=0.2, color='green', label='Peak Hours (6-9 PM)')
plt.legend()

plt.tight_layout()
plt.show()

best_hour = hours[np.argmax(predictions_by_hour)]
print(f"\nBest publishing hour: {best_hour}:00 ({best_hour % 12 or 12} {'PM' if best_hour >= 12 else 'AM'})")
print(f"Expected views at best hour: {max(predictions_by_hour):,.0f}")
print(f"Expected views at worst hour: {min(predictions_by_hour):,.0f}")
print(f"Difference: {max(predictions_by_hour) - min(predictions_by_hour):,.0f} views")

## 9. Get Recommendations

In [ ]:
# Get optimization recommendations
recommendations = get_optimal_features()

print("Optimization Recommendations:")
print("="*50)

print("\n📝 Title Recommendations:")
for rec in recommendations['title_recommendations']:
    print(f"  • {rec}")

print("\n⏰ Temporal Recommendations:")
for rec in recommendations['temporal_recommendations']:
    print(f"  • {rec}")

print("\n🎬 Duration Recommendations:")
for rec in recommendations['duration_recommendations']:
    print(f"  • {rec}")

print("\n🏷️ Metadata Recommendations:")
for rec in recommendations['metadata_recommendations']:
    print(f"  • {rec}")

## 10. Save Model

In [ ]:
# Save the trained model
predictor.save_model(model_dir='../models')
print("Model saved successfully!")

## Summary

This notebook demonstrated:
- How to load and explore YouTube video data
- Feature extraction and engineering process
- Training a machine learning model for view prediction
- Analyzing feature importance
- Making predictions for new videos
- Understanding optimization strategies

**Key Findings:**
1. Publishing during peak hours (6-9 PM) significantly increases views
2. Weekend uploads generally perform better
3. Title optimization (length, questions, numbers) matters
4. Optimal video duration is 7-15 minutes for most content
5. Using 10-15 relevant tags improves discoverability

**Next Steps:**
- Collect real YouTube data using the API
- Retrain model with your specific data
- Test predictions against actual results
- Iterate and improve based on learnings